# Guest Lecture: Lessons Learned Building the Neural GPU
*Łukasz Kaiser*  
build NNs to learn algorithms

## 1. Feed-forward vs. Recurrent Neural Network
- Feed-forward neural network
    - fixed-size input
    - fixed-size output
    - number of parameters depends on these sizes
    - limited computational power
- RNN
    - variable-size input
    - variable-size output
    - number of parameters depends on memory size
    - computational power limited by this memory size
 

## 2. Neural Algorithm Learning
### 2.1 Previous work on neural algorithm learning
- RNN and LSTM-based, generalizes a bit with attention
- Data-structure based (stack RNN, memory nets)
- Neural Turing Machines

### 2.2 problem with RNN
- feasible only if testing length ~ training
- with higher length we need to increase network capacity (number of parameters)
- Fail to capture algorithmic patterns, even in principle

### 2.3 problem with NTM
- Look close, your computer does not have a tape!
- Very sequential in nature, hard to train on hard problems
- Example problem: long multiplication


## 3. Neural Computation
### 3.1 RNN vs NTM
- Recurrent Neural Networks: share weights across time steps but use fixed-size memory vectors
    - Time: $O(f(n))$
    - Space: $O(1)$
- Neural Turing Machines:  work with variable-size memory and so can do general computation, but are sequential
    - Time: $O(f(n))$
    - Space: $O(g(n))$
    
![RNNvsNTM](figures/12_01.png)

### 3.2 Arbitrary Size Ops
Neural computation requires operators that use a **fixed number of parameters** but operate on **memory of arbitrary size**.  
Candidate operators:
- Attention (Neural Turing Machine)  
- Stack, (De)Queue, Lists
- ... other memory structures ...
- **Convolutions!**: act like a **neural GPU**
    - Attention is a softmax, effectively ~1 memory item / step.
    - Similarly a stack and other task-specific memory structures
    - Convolutions affect <u>all memory items in each step</u>
    - Convolutions are already implemented and optimized
    - To train well we need to use LSTM-style gates: <u>CGRN</u> (Convolutional Gated Recurrent Networks)
![attention](figures/12_02.png)


## 4. Neural GPU
**Convolutional Gated Recurrent Networks (CGRNs)** perform many parallel operations in each step, akin
to a neural GPU and in contrast to the sequential nature of Neural Turing Machines.  
### 4.1 CGRU definition
(similar to GRU replacing linear by convolution)  
$$s_{t+1}=g \cdot s_t + (1-g) \cdot c$$  
where:  
$$g=sigmoid(conv(s_t, K_g))$$  
$$c=tanh(conv(s_t \cdot r, K_c))$$  
$$r=sigmoid(conv(s_t, K_r))$$

### 4.2 Computational power
- Small number of parameters (3 kernels: $K_g$, $K_c$, $K_r$)
- Can simulate computation of **cellular automata**
- With memory of size $n$ can do $n$ local operations / step
- E.g., can do long multiplication in $O(n)$ steps  
![CGRU](figures/12_03.png)

### 4.3 Results
A n-step n-size memory CGRN achieves 100x generalization on a number of tasks such as reversing sequences, long addition, or even non-linear-time long multiplication.  
![results](figures/12_04.png)

### 4.4 Tricks of the Trade
#### Curriculum learning
- Start training on small lengths, increase when learned  

#### Parameter sharing relaxation
- Allow to do different things in different time-steps first
- Not needed now with bigger models and orthogonal init  

#### Dropout on recurrent connections
- Randomly set 10% of state vectors to 0 in each step
- Interestingly, this is key for good length generalization  

#### Noise added to gradients
- Add small gaussian noise to gradients in each training step  

#### Gate cutoff (saturation)
- Instead of $sigmoid(x)$ use $[1.2sigmoid(x) - 0.1]_{[0,1]}$  

#### Tune parameters
- A lot of GPUs running for a few months; or: better method!

### 4.5 How to code
#### Is the graph static or dynamic?
- Dynamic ops were not exposed in first TF release, tricky
- Static graph requires bucketing and takes long to build
- **Focus**: why conditionals are tricky: batches and lambdas  

#### How do we do bucketing?
- Sparse or dense buckets? Masks are bug-prone.
- How do we bucket training data? Feed or queues?
- If queues, how to do curriculum? How to not starve?  

#### How do we write layers?
- Is there a canonical way to define new functions?
- Frameworks: Keras vs Slim (OO vs functional)
- Unification in `tf.layers`: callable objects save scope
- Example: weight-normalization through custom_getter  

#### How do we organize experiments?
- Use `tf.learn`, Estimator, Experiment.
- How to register models and problems? Save runs?
- Hyper-parameters manual or tuned with ranges?